In [86]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import seaborn as sns
import openpyxl
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

In [87]:
pyt_lical = r'C:\PyProjects\poject_1\tural\site'

In [88]:
pyt_orders = pyt_lical+'/orders11.xls'
pyt_sebes = pyt_lical+'/sebestoim.xlsx'

pyt_zapros = pyt_lical+'/tovarn_zapros.xlsx'
pyt_result = pyt_lical+'/tovarn_result.xlsx'

pyt_base_nabor = pyt_lical+'/base_nabor.xlsx'

In [89]:
df_orders = pd.read_excel(pyt_orders)
df_sebes = pd.read_excel(pyt_sebes)

In [90]:
df_orders['Дата и время'] = pd.to_datetime(df_orders['Дата и время'], format='%d.%m.%Y %H:%M')
df_orders = df_orders.merge(df_sebes, left_on='Артикул', right_on='sku', how = 'left')
df_orders = df_orders.drop('sku', axis=1)
df_tovarn = df_orders[['Номер', 'Дата и время','Название', 'Артикул', 'Количество', 'Общая сумма товара', 'sebes']]
df_tovarn['Общая себестоимость товара'] = df_tovarn['Количество']*df_tovarn['sebes']
df_tovarn = df_tovarn.drop('sebes', axis=1)

df_orderswithdates = df_tovarn.drop(['Номер'], axis=1)
df_orderswithdates.columns =['Дата','Название','Артикул','Количество','Выручка','Себестоимость']
df_orderswithdates = df_orderswithdates[['Артикул','Название','Дата','Количество','Выручка','Себестоимость']]
df_orderswithdates['Маржа']= df_orderswithdates['Выручка']-df_orderswithdates['Себестоимость']
df_orderswithdates['Маржинальность']= (df_orderswithdates['Выручка']-df_orderswithdates['Себестоимость'])/df_orderswithdates['Выручка']

df_tovarn['Месяц'] = df_tovarn['Дата и время'].dt.month
df_tovarn['День'] = df_tovarn['Дата и время'].dt.day

df_to_nabor = df_tovarn

df_tovarn = df_tovarn.drop(['Номер', 'Дата и время'], axis=1)
df_tovarn_group = df_tovarn.groupby(['Артикул','Название', 'Месяц'])['Количество','Общая сумма товара','Общая себестоимость товара'].agg({"sum"}).reset_index().sort_values(('Месяц'), ascending =1)
df_tovarn_group.columns =['Артикул','Название','Месяц','Количество','Выручка','Себестоимость']
df_tovarn_group['Маржа']= df_tovarn_group['Выручка']-df_tovarn_group['Себестоимость']
df_tovarn_group['Маржинальность']= (df_tovarn_group['Выручка']-df_tovarn_group['Себестоимость'])/df_tovarn_group['Выручка']

In [91]:
list_naborov = ['nb-vse']

In [92]:
df_tovarn_nab = df_to_nabor.query("Артикул in @list_naborov")
df_tovarn_NE_nab = df_to_nabor.query("Артикул not in @list_naborov")

In [93]:
df_base_nabor = pd.read_excel(pyt_base_nabor)
df_base_nabor['date_1'] = pd.to_datetime(df_base_nabor['date_1'], format='%Y%m%d')
df_base_nabor['date_2'] = pd.to_datetime(df_base_nabor['date_2'], format='%Y%m%d')
df_base_nabor

,sku_1c,sku_nab,seb,vyr,date_1,date_2
0,nb-vse,mk-1-6(nb),0.563981,0.601464,2000-11-15,2022-11-09
1,nb-vse,om-bfd-369-90(nb),0.108610,0.096278,2000-11-15,2022-11-09
2,nb-vse,bt-bfd-10000-90(nb),0.165087,0.143868,2000-11-15,2022-11-09
3,nb-vse,zk-bfd-120(nb),0.071090,0.097498,2000-11-15,2022-11-09
4,nb-vse,(nb),0.090442,0.060891,2000-11-15,2022-11-09
5,nb-vse,mk-1-6(nb),0.575810,0.600000,2022-11-09,2022-11-21
6,nb-vse,om-bfd-369-90(nb),0.110890,0.096040,2022-11-09,2022-11-21
7,nb-vse,bt-now-5000(nb),0.147580,0.145950,2022-11-09,2022-11-21
8,nb-vse,zk-bfd-120(nb),0.072580,0.097260,2022-11-09,2022-11-21
9,nb-vse,(nb),0.092340,0.060740,2022-11-09,2022-11-21


In [99]:
df_base_nabor_merge = df_base_nabor.merge(df_tovarn_nab, left_on='sku_1c', right_on='Артикул', how = 'left')
df_base_nabor_merge['Общая сумма товара'] = df_base_nabor_merge['Общая сумма товара']*df_base_nabor_merge['vyr']
df_base_nabor_merge['Общая себестоимость товара'] = df_base_nabor_merge['Общая себестоимость товара']*df_base_nabor_merge['seb']
df_base_nabor_merge = df_base_nabor_merge.rename(columns={'Дата и время': 'Дата_и_время'})

df_base_nabor_merge = df_base_nabor_merge.query('Дата_и_время >= date_1 and Дата_и_время <= date_2')
df_base_nabor_merge = df_base_nabor_merge.rename(columns={'Дата_и_время': 'Дата и время'})

df_base_nabor_merge = df_base_nabor_merge[['Номер', 'Дата и время', 'Название', 'sku_nab','Количество', 'Общая сумма товара','Общая себестоимость товара', 'Месяц', 'День']]
df_base_nabor_merge = df_base_nabor_merge.rename(columns={'sku_nab': 'Артикул'})

In [106]:
df_base_s_zamenoi_naborov = pd.concat([df_tovarn_NE_nab, df_base_nabor_merge])

In [107]:
def svod_marga(df):
    df = df.drop(['Номер', 'Дата и время'], axis=1)
    df_group = df.groupby(['Артикул','Название', 'Месяц'])['Количество','Общая сумма товара','Общая себестоимость товара'].agg({"sum"}).reset_index().sort_values(('Месяц'), ascending =1)
    df_group.columns =['Артикул','Название','Месяц','Количество','Выручка','Себестоимость']
    df_group['Маржа']= df_group['Выручка']-df_group['Себестоимость']
    df_group['Маржинальность']= (df_group['Выручка']-df_group['Себестоимость'])/df_group['Выручка']
    return df_group

In [108]:
df_base_s_zamenoi_naborov_svod =  svod_marga(df_base_s_zamenoi_naborov)
df_base_s_zamenoi_naborov_svod

,Артикул,Название,Месяц,Количество,Выручка,Себестоимость,Маржа,Маржинальность
432,kps-328,Kapous Укрепляющая сыворотка с биотином для ст...,9,2,1198.0,818.00,380.00,0.317195
376,iron100,"Smartlife Липосомальное железо (флакон, 100мл)...",9,1,900.0,630.00,270.00,0.300000
756,trs-oil2,Trius / масло для бороды,9,1,1199.0,600.00,599.00,0.499583
370,gls-73334,"GLS Витамины для волос, 60 капс",9,9,5841.0,3060.00,2781.00,0.476117
367,gls-73297,"GLS Биотин без сахара со вкусом вишни, 20 шипу...",9,1,459.0,226.67,232.33,0.506166
...,...,...,...,...,...,...,...,...
431,kps-327,Kapous Маска с биотином для укрепления и стиму...,11,1,934.0,549.00,385.00,0.412206
429,kps-326,Kapous Лосьон с биотином для укрепления и стим...,11,1,899.0,429.00,470.00,0.522803
426,kps-325,Kapous Шампунь с биотином для укрепления и сти...,11,5,3854.0,1845.00,2009.00,0.521277
420,kps-2898,Kapous Professional Увлажняющий мужской гель д...,11,1,1189.0,659.00,530.00,0.445753


In [6]:
def save(df,pyt):
    with pd.ExcelWriter(pyt,engine="openpyxl",mode='w') as writer:
        df.to_excel(writer,index=False)

In [7]:
# Сохранение общей товарной аналитики
save(df_tovarn_group,pyt_result)

In [8]:
# Расчёт наценки
month = 10
marja = 10000
isklucheniya = []
#list(set(df_tovarn_group['Артикул'].to_list()))
vyborka = list(set(df_tovarn_group['Артикул'].to_list()))

df_mesyach = df_tovarn_group.query("Месяц == @month and Артикул not in @isklucheniya and Артикул in @vyborka")
sebes = df_mesyach['Себестоимость'].sum()
vyr = df_mesyach['Выручка'].sum()
nacenka = (vyr-sebes)/sebes*100
nacenka
#Таблица всех sku, участвующих в расчёте.
#df_mesyach.sort_values(('Маржа'),ascending=0).reset_index().head(75)

213.1966339351417

In [49]:
#Spiski
biofoods = ['zk-bfd-120','zk-bfd-240','bt-bfd-5000-90','bt-bfd-10000-90','mc1-bfd-90','om-bfd-369-90','snh-bfd-90',
            'om-bfd-3-90','bcx-bfd-90','wc1-bfd-90']
xinox = ['xn-5-1','xn-5-2','xn-5-3','xn-5-4','xn-5-6-k','xn-5-12','xn-15-1','xn-15-3','xn-15-6','xn-2-1','xn-2-2','xn-2-3',
         'xn-5','xn-15']
natrol = ['nat-comb-90t','bt-ntrl-10000','nat-mc','nat-gluk-90','nat-ala1','nat-shn-60c','nat-b12-5000-100t','nat-d3-50-90t','nat-d3-125-90t','bt-ntrl-5000','nat-ham','nat-tcla']
pal = ['swn-d3-5000-250s','swn-crp-200-100c','swn-ker-50-60c']

In [28]:
def spisokanalytics(spisok):
    df_spisok = df_tovarn_group[df_tovarn_group['Артикул'].isin(spisok)]
    return(df_spisok)

In [50]:
# Для конкретной товарной группы
df_tovarn_zapros = spisokanalytics(pal)
df_tovarn_zapros

,Артикул,Название,Месяц,Количество,Выручка,Себестоимость,Маржа,Маржинальность
711,swn-d3-5000-250s,"Swanson Витамин D3 5000ME, 250 жевательных таб...",11,4,8496.0,1866.88,6629.12,0.780264
710,swn-crp-200-100c,"Swanson Пиколинат хрома 200mcg, 100 капсул",11,1,977.0,0.00,977.00,1.000000
715,swn-ker-50-60c,"Swanson Кератин 50мг, 60 капсул",11,1,1399.0,0.00,1399.00,1.000000


In [ ]:
with pd.ExcelWriter(pyt_zapros,engine="openpyxl",mode='w') as writer:
        df_tovarn_zapros.to_excel(writer, sheet_name='vse',index=False)

Вычисление доли в наборах

In [18]:
period_biofoods =[['20001115', '20221109'],['20221109', '20221121'],['20221121', '20221221'],['20221221', '20251109']]
period_xinox = [['20001115','20251109']]
sku_nabor_biofoods = ['nb-vse','nb-eco','nb-poln','nb-vitamin','nb-optim','nb-bystr','nb-gensk','nb-xn6','nb-zvrsh']
sku_nabor_xinox = ['nb-poln','nb-bystr','nb-gensk','nb-shag','nb-xn6']

koef_nabor_biofoods = {'nb-vse':[[0.33,0.34],[0.19,0.18],[0.24,0.27],[0.33,0.34]],
              'nb-eco':[[0.25,0.19],[0.25,0.19],[0,0],[0.25,0.19]],
              'nb-poln':[[0.24,0.41],[0.15,0.26],[0.24,0.41],[0.24,0.41]],
              'nb-vitamin':[[0.8,0.89],[0.56,0.59],[0.63,0.76],[0.8,0.89]],
              'nb-optim':[[0.33,0.34],[0.13,0.1],[0.19,0.24],[0.33,0.34]],
              'nb-bystr':[[0.28,0.43],[0.13,0.14],[0.15,0.29],[0.28,0.43]],
              'nb-gensk':[[0.63,0.77],[0.33,0.42],[0.63,0.77],[0.63,0.77]],
              'nb-xn6':[[0.23,0.3],[0.11,0.1],[0.12,0.2],[0.23,0.3]],
              'nb-zvrsh':[[0.49,0.59],[0,0],[0.49,0.59],[0.49,0.59]],
              }
koef_nabor_xinox = {'nb-poln':[[0.56363,0.25853]],
              'nb-bystr':[[0.47952,0.18285]],
              'nb-gensk':[[0.33154,0.17119]],
              'nb-shag':[[0.51893,0.33259]],
              'nb-xn6':[[0.52884,0.55972]]
              }
def dfart(df,sku,period):
    return df.query("Артикул == @sku and Дата > @period[0] and Дата <= @period[1]" )

def pereschet(df,koef):
    df['Выручка'] = df['Выручка']*koef[0]
    df['Себестоимость'] = df['Себестоимость']*koef[1]
    df['Маржа']= df['Выручка']-df['Себестоимость']
    df['Маржинальность']= (df['Выручка']-df['Себестоимость'])/df['Выручка']
    return df

def tov_is_nabora(df,period,koef,sku_all):
    df_all = pd.DataFrame()
    z = 0
    for sku in sku_all:
        j = 0
        for i in period:
            df_2 = dfart(df,sku,i)
            df_3 = pereschet(df_2,koef[sku][j])
            df_all = pd.concat([df_all,df_3], ignore_index=True)
            j += 1
        z+=1
    df_all['Месяц'] = df_all['Дата'].dt.month
    df_all = df_all.drop('Дата', axis=1)
    df_all = df_all[['Артикул', 'Название','Месяц','Количество', 'Выручка', 'Себестоимость', 'Маржа', 'Маржинальность' ]]
    return df_all.query('Выручка > 0')

def brand_group(df,df2):
    df_obsh = pd.concat([df, df2], ignore_index=True)
    df_obsh = df_obsh.groupby(['Артикул','Название', 'Месяц']).agg({'Количество':'sum','Выручка':'sum','Себестоимость':'sum','Маржа':'sum','Маржинальность':'mean'}).reset_index().sort_values(('Месяц'), ascending =1)
    df_obsh[['Выручка', 'Себестоимость', 'Маржа']] = df_obsh[['Выручка', 'Себестоимость', 'Маржа']].astype(int)
    df_obsh['Маржинальность'] = df_obsh['Маржинальность'].round(3)
    return df_obsh

def svod_group(df):
    df = df.groupby(['Месяц']).\
        agg({'Количество':'sum','Выручка':'sum','Себестоимость':'sum','Маржа':'sum','Маржинальность':'mean'}).\
        reset_index()
    df[['Выручка', 'Себестоимость', 'Маржа']] = df[['Выручка', 'Себестоимость', 'Маржа']].astype(int)
    df['Маржинальность'] = df['Маржинальность'].round(3)
    return df

In [10]:
#biofoods
df_biofoods_v_naborah = tov_is_nabora(df_orderswithdates,period_biofoods,koef_nabor_biofoods,sku_nabor_biofoods)
df_biofoods_i_nabor = brand_group(spisokanalytics(biofoods),df_biofoods_v_naborah)
save(df_biofoods_i_nabor,pyt_result)

In [11]:
svod_group(df_biofoods_i_nabor)

,Месяц,Количество,Выручка,Себестоимость,Маржа,Маржинальность
0,9,669,658653,204694,453954,0.629
1,10,966,935139,292041,643093,0.639
2,11,610,531428,166703,364721,0.648


In [19]:
#xinox
df_xinox_v_naborah = tov_is_nabora(df_orderswithdates,period_xinox,koef_nabor_xinox,sku_nabor_xinox)
df_xinox_i_nabor = brand_group(spisokanalytics(xinox),df_xinox_v_naborah)
save(df_xinox_i_nabor,pyt_result)

In [20]:
svod_group(df_xinox_i_nabor)

,Месяц,Количество,Выручка,Себестоимость,Маржа,Маржинальность
0,9,807,1564841,287096,1277738,0.810
1,10,1024,2167893,395530,1772358,0.812
2,11,834,1670248,327122,1343120,0.798
